In [213]:
import pandas as pd
from sklearn import metrics, datasets
import numpy as np
import random

pd.set_option('mode.chained_assignment', None)
# This code will not complain!

iris = datasets.load_iris()
irisDF = pd.DataFrame(data= np.c_[iris['data'], iris['target']], columns= iris['feature_names'] + ['target'])

irisDF['target_name'] = ''

for i in irisDF.index:
    irisDF.target_name[i] = iris['target_names'][int(irisDF.target[i])]

irisDF = irisDF.sample(frac=1,random_state=1).reset_index(drop=True)

trainIris = irisDF[:110]

testIris = irisDF[110:].reset_index(drop=True)
testIris['hyp'] = np.nan

testIris


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target,target_name,hyp
0,5.6,2.8,4.9,2.0,2.0,virginica,NaN
1,6.8,3.2,5.9,2.3,2.0,virginica,NaN
2,5.0,3.3,1.4,0.2,0.0,setosa,NaN
3,5.1,3.7,1.5,0.4,0.0,setosa,NaN
4,5.9,3.2,4.8,1.8,1.0,versicolor,NaN
5,4.6,3.1,1.5,0.2,0.0,setosa,NaN
6,5.8,2.7,5.1,1.9,2.0,virginica,NaN
7,4.8,3.1,1.6,0.2,0.0,setosa,NaN
8,6.5,3.0,5.2,2.0,2.0,virginica,NaN
9,4.9,2.5,4.5,1.7,2.0,virginica,NaN


In [214]:
def stats(test):
    
    pasBon = 0
    
    bonParClasse = {}
    for c in test.target.unique():
            bonParClasse[c] = 0

    for i in test.index:

        if (test.target[i] == test.hyp[i]):
            bonParClasse[test.hyp[i]] += 1
        else:
            pasBon += 1

    #Calcul précision
    precision = 0
    for c in bonParClasse.keys():
        precision += bonParClasse[c] / test.hyp.value_counts

    precision /= len(test)

    return {'Erreur': pasBon/(len(test)), 'Precision': precision}

def stats2(test):

    print(metrics.classification_report(test.target, test.hyp, target_names = test.target_name.unique(), zero_division=1))


In [215]:
def trainZeroR(train):
    return np.argmax(np.bincount(train.target))
    
def testIrisZeroR(test, model):
    for i in test.index:
        test.hyp[i] = model

In [216]:
model = trainZeroR(trainIris)

testIrisZeroR(testIris, model)

stats2(testIris)
testIris



              precision    recall  f1-score   support

   virginica       1.00      0.00      0.00        14
      setosa       1.00      0.00      0.00        14
  versicolor       0.30      1.00      0.46        12

    accuracy                           0.30        40
   macro avg       0.77      0.33      0.15        40
weighted avg       0.79      0.30      0.14        40



,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target,target_name,hyp
0,5.6,2.8,4.9,2.0,2.0,virginica,2.0
1,6.8,3.2,5.9,2.3,2.0,virginica,2.0
2,5.0,3.3,1.4,0.2,0.0,setosa,2.0
3,5.1,3.7,1.5,0.4,0.0,setosa,2.0
4,5.9,3.2,4.8,1.8,1.0,versicolor,2.0
5,4.6,3.1,1.5,0.2,0.0,setosa,2.0
6,5.8,2.7,5.1,1.9,2.0,virginica,2.0
7,4.8,3.1,1.6,0.2,0.0,setosa,2.0
8,6.5,3.0,5.2,2.0,2.0,virginica,2.0
9,4.9,2.5,4.5,1.7,2.0,virginica,2.0


In [217]:
def discretize(data, means, featuresToDiscretize):
    
    dataDiscret = data.copy()
    
    for col in featuresToDiscretize:
        for i in data.index:
            
            if (data[col][i] >= means[col]):
                dataDiscret[col][i] = "long"
            else:
                dataDiscret[col][i] = "court"
                
    return dataDiscret


trainIrisDis = discretize(trainIris, trainIris.mean(), iris['feature_names'])
testIrisDis = discretize(testIris, trainIris.mean(), iris['feature_names'])


testIrisDis

C:\Users\romai\AppData\Local\Temp\ipykernel_12980\1767731455.py:16: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  trainIrisDis = discretize(trainIris, trainIris.mean(), iris['feature_names'])
C:\Users\romai\AppData\Local\Temp\ipykernel_12980\1767731455.py:17: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  testIrisDis = discretize(testIris, trainIris.mean(), iris['feature_names'])


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target,target_name,hyp
0,court,court,long,long,2.0,virginica,2.0
1,long,long,long,long,2.0,virginica,2.0
2,court,long,court,court,0.0,setosa,2.0
3,court,long,court,court,0.0,setosa,2.0
4,court,long,long,long,1.0,versicolor,2.0
5,court,long,court,court,0.0,setosa,2.0
6,court,court,long,long,2.0,virginica,2.0
7,court,long,court,court,0.0,setosa,2.0
8,long,court,long,long,2.0,virginica,2.0
9,court,court,long,long,2.0,virginica,2.0


In [218]:
def trainOneR(train, featuresColumns):
    
    bestPrecision = -1

    for col in featuresColumns:
        res = pd.DataFrame(data = np.zeros((len(train.target.unique()), len(train[col].unique()))), columns = train[col].unique())

        for i in train.index:
            res[train[col][i]][int(train.target[i])] += 1
        
        precision = res.max().sum() / res.sum().sum()

        #Update du model avec nouvelle colone
        if (precision > bestPrecision):
            bestPrecision = precision
            model = {'feature_name': col}
            
            for value in res:
                print()
                model[value] = res.idxmax()[value]

    return model



In [219]:
def testOneR(test, model):
    for i in test.index:
        test.hyp[i] = model[ test[ model['feature_name'] ][i] ]

model = trainOneR(trainIrisDis, iris["feature_names"])
    
testOneR(testIrisDis, model)
testIrisDis.hyp = np.nan

testOneR(testIrisDis, model)
stats2(testIrisDis)
testIrisDis







              precision    recall  f1-score   support

   virginica       0.78      1.00      0.88        14
      setosa       1.00      0.00      0.00        14
  versicolor       0.55      1.00      0.71        12

    accuracy                           0.65        40
   macro avg       0.77      0.67      0.53        40
weighted avg       0.79      0.65      0.52        40



,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target,target_name,hyp
0,court,court,long,long,2.0,virginica,2.0
1,long,long,long,long,2.0,virginica,2.0
2,court,long,court,court,0.0,setosa,0.0
3,court,long,court,court,0.0,setosa,0.0
4,court,long,long,long,1.0,versicolor,2.0
5,court,long,court,court,0.0,setosa,0.0
6,court,court,long,long,2.0,virginica,2.0
7,court,long,court,court,0.0,setosa,0.0
8,long,court,long,long,2.0,virginica,2.0
9,court,court,long,long,2.0,virginica,2.0


In [220]:
def trainNaiveBayes(train, featuresColumns):
    
    featuresRes = {}
    bayesProbs = {}
    classesProbs = {}

    for c in train.target.unique():

        classesProbs[c] = pd.value_counts(train.target)[c] / train.target.count()

    for col in featuresColumns:
        res = pd.DataFrame(data = np.ones((len(train.target.unique()), len(train[col].unique()))), columns = train[col].unique())
        
        for i in train.index:
            res[train[col][i]][int(train.target[i])] += 1
        
        featuresRes[col] = res


    for feature in featuresRes.keys():
        resProbs = featuresRes[feature].copy()

        for ind in featuresRes[feature].index:
            for fValueName in train[col].unique():
                resProbs[fValueName][ind] = featuresRes[feature][fValueName][ind] / featuresRes[feature].sum(1)[ind]
        bayesProbs[feature] = resProbs


    model = {'featuresProbs': bayesProbs, 'classesProbs': classesProbs}

    return model

In [221]:
def testNaiveBayes(test, model):

    for i in test.index:

        probs = {}
        for c in model['classesProbs'].keys():

            prob=1
            for f in model['featuresProbs'].keys():
                prob *=   model['featuresProbs'][f][test[f][i]][c]
            
            prob *= model['classesProbs'][c]
            probs[c] = prob

        test.loc[test.hyp][i] = max(probs, key=probs.get)

    print('ok')
    


In [222]:


model = trainNaiveBayes(trainIrisDis, iris["feature_names"])

stats2(testIrisDis)

testNaiveBayes(testIrisDis, model)

testIrisDis


              precision    recall  f1-score   support

   virginica       0.78      1.00      0.88        14
      setosa       1.00      0.00      0.00        14
  versicolor       0.55      1.00      0.71        12

    accuracy                           0.65        40
   macro avg       0.77      0.67      0.53        40
weighted avg       0.79      0.65      0.52        40

ok


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target,target_name,hyp
0,court,court,long,long,2.0,virginica,2.0
1,long,long,long,long,2.0,virginica,2.0
2,court,long,court,court,0.0,setosa,0.0
3,court,long,court,court,0.0,setosa,0.0
4,court,long,long,long,1.0,versicolor,2.0
5,court,long,court,court,0.0,setosa,0.0
6,court,court,long,long,2.0,virginica,2.0
7,court,long,court,court,0.0,setosa,0.0
8,long,court,long,long,2.0,virginica,2.0
9,court,court,long,long,2.0,virginica,2.0


In [223]:
def distMan(dataX, dataY, featuresNames):
    dist = 0
    for f in featuresNames:
        dist += np.abs(dataY[featuresNames][0] - dataX[featuresNames][0])

    return dist

In [224]:
from heapq import nsmallest

def KPPV(test, train, featuresNames, k):

    for testInd in test.index:
        distances = {}
        for trainInd in train.index:
            distances[trainInd] = distMan(test.iloc[testInd], train.iloc[trainInd], featuresNames)
        
        closestIndex = nsmallest(k, distances, key = distances.get)
        
        closestClasses = []

        for trainInd in closestIndex:
            closestClasses.append(train.iloc[trainInd].target)
        test.hyp[testInd] = max(closestClasses,key=closestClasses.count)

In [225]:
KPPV(testIris, trainIris, iris['feature_names'], 8)

In [226]:


stats2(testIris)

testIris

              precision    recall  f1-score   support

   virginica       0.82      1.00      0.90        14
      setosa       0.62      0.57      0.59        14
  versicolor       0.60      0.50      0.55        12

    accuracy                           0.70        40
   macro avg       0.68      0.69      0.68        40
weighted avg       0.68      0.70      0.69        40



,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target,target_name,hyp
0,5.6,2.8,4.9,2.0,2.0,virginica,1.0
1,6.8,3.2,5.9,2.3,2.0,virginica,2.0
2,5.0,3.3,1.4,0.2,0.0,setosa,0.0
3,5.1,3.7,1.5,0.4,0.0,setosa,0.0
4,5.9,3.2,4.8,1.8,1.0,versicolor,1.0
5,4.6,3.1,1.5,0.2,0.0,setosa,0.0
6,5.8,2.7,5.1,1.9,2.0,virginica,1.0
7,4.8,3.1,1.6,0.2,0.0,setosa,0.0
8,6.5,3.0,5.2,2.0,2.0,virginica,1.0
9,4.9,2.5,4.5,1.7,2.0,virginica,0.0
